## Célula de leitura do arquivo

In [135]:
'''
  def ler_arquivo(nome_arquivo):
      try:
          with open(nome_arquivo, "r") as f:
              linhas = f.readlines()
          return linhas
      except FileNotFoundError:
          print("Arquivo de entrada não encontrado!")
          return []


  linhas_arquivo = ler_arquivo("testes.txt")
'''

'\n  def ler_arquivo(nome_arquivo):\n      try:\n          with open(nome_arquivo, "r") as f:\n              linhas = f.readlines()\n          return linhas\n      except FileNotFoundError:\n          print("Arquivo de entrada não encontrado!")\n          return []\n\n\n  linhas_arquivo = ler_arquivo("testes.txt")\n'

## Conjunto de Regras

In [136]:
regras = {
    "FORMULA": "CONSTANTE | PROPOSICAO | FORMULAUNARIA | FORMULABINARIA",
    "CONSTANTE": "true | false",
    "PROPOSICAO": "[0-9][0-9a-z]*",
    "FORMULAUNARIA": "( OPERATORUNARIO FORMULA )",
    "FORMULABINARIA": "( OPERATORBINARIO FORMULA FORMULA )",
    "OPERATORUNARIO": "\\neg",
    "OPERATORBINARIO": "\\wedge | \\vee | \\rightarrow | \\leftrightarrow",
    "ABREPAREN": "(",
    "FECHAPAREN": ")"
}



In [137]:
# Funções auxiliares
# Extrai operadores e constantes
operadores_binarios = [op.strip() for op in regras["OPERATORBINARIO"].split("|")]
operadores_unarios = [regras["OPERATORUNARIO"]]
constantes = [c.strip() for c in regras["CONSTANTE"].split("|")]

# Verifica se é uma proposição válida
def eh_proposicao(token):
    if not token or not token[0].isdigit():
        return False
    for c in token[1:]:
        if not (c.isdigit() or 'a' <= c <= 'z'):
            return False
    return True

# Verifica se é constante
def eh_constante(token):
    return token in constantes

In [138]:
class StateMachine:
    def __init__(self):
        self.state = 'start'

        self.transitions = {
            'start': {'CONSTANTE': 'formula_simples_correta',
                      'PROPOSICAO': 'formula_simples_correta',
                      'ABREPAREN': 'abre_parenteses'},

            'formula_simples_correta': {' ': 'formula_simples_correta'},

            'abre_parenteses': {' ': 'abre_parenteses',
                                'ABREPAREN': 'abre_parenteses',
                                'OPERADORUNARIO': 'operador_unario',
                                'OPERADORBINARIO': 'operador_binario',},

            'operador_unario': {' ': 'operador_unario',
                                'CONSTANTE': 'termo_unario',
                                'PROPOSICAO': 'termo_unario',
                                'ABREPAREN': 'abre_parenteses'},

            'termo_unario': {' ': 'termo_unario',
                             'FECHAPAREN': 'fecha_parenteses'},

            'operador_binario': {' ': 'operador_binario',
                                 'ABREPAREN': 'abre_parenteses',
                                 'CONSTANTE': 'termo_binario_nao_final',
                                 'PROPOSICAO': 'termo_binario_nao_final' },

            'termo_binario_nao_final': {' ': 'termo_binario_nao_final',
                                        'CONSTANTE': 'termo_binario_final',
                                        'PROPOSICAO': 'termo_binario_final',
                                        'ABREPAREN': 'abre_parenteses'},

            'termo_binario_final': {' ': 'termo_binario_final',
                                        'FECHAPAREN': 'fecha_parenteses'},

            'fecha_parenteses': {' ': 'fecha_parenteses',
                                 'ABREPAREN': 'abre_parenteses',
                                 'FECHAPAREN' : 'fecha_parenteses',
                                 'CONSTANTE' : 'termo_binario_final',
                                 'PROPOSICAO': 'termo_binario_final'}
        }

        self.final_states = {'formula_simples_correta', 'fecha_parenteses'}

    def trigger(self, event):
        if event in self.transitions[self.state]:
            new_state = self.transitions[self.state][event]
            self.state = new_state

    def process(self, tokens):
      self.state = 'start'

      tokens_converted = [] # convertendo cada token para simplificar o processo da maquina de estados
      for token in tokens:
        if eh_constante(token):
          tokens_converted.append('CONSTANTE')

        elif token == '(':
          tokens_converted.append('ABREPAREN')
        elif token == ')':
          tokens_converted.append('FECHAPAREN')

        elif token == '\\neg':
          tokens_converted.append('OPERADORUNARIO')

        elif token == '\\wedge' or token == "\\vee" or token == "\\rightarrow" or token == "\\leftrightarrow":
          tokens_converted.append('OPERADORBINARIO')

        elif eh_proposicao(token):
          tokens_converted.append('PROPOSICAO')

        else:
          tokens_converted.append(token)

      for token in tokens_converted:
          if token not in self.transitions.get(self.state, {}):
              return False
          self.trigger(token)

      if(self.state in self.final_states):
        return tokens

In [139]:
# Parser usando uma tabela de parsing

class LL1_Parser:

    # Iniciando a classe com os tokens, inserindo $ na ultima posição para validar o fim
    def __init__(self, tokens):
        self.tokens = tokens + ['$']
        self.pos = 0

    # Métodos para progredir na lista de tokens:
    # pegar a token atual
    # ir para a proxima token
    # Método que vai para a proxima token ao encontrar o valor esperado na lista de tokens
    def atual(self):
        return self.tokens[self.pos]

    def proximo(self):
        self.pos += 1

    def match(self, esperado):
        # print("MATCH: " + esperado)
        if self.atual() == esperado:
            self.proximo()

    # Método que inicia o parser com a regra inicial F (FORMULA)
    def parse(self):
        self.F()
        if self.atual() != '$':
            return False
        return True

    # Métodos para tratar cada regra de produção
    # o método verifica o token atual e utiliza a regra necessária conforme o token identificado
    # se nenhuma regra for utilizada para algum token então o proximo token não vai ser chamado e a validação não vai chegar em '$'

    def F(self):
        token = self.atual()

        if eh_constante(token):
            self.C()
        elif eh_proposicao(token):
            self.P()
        elif token == '(':
            lookahead = self.tokens[self.pos + 1]
            if lookahead == '\\neg':
                self.FU()
            elif lookahead in {'\\wedge', '\\vee', '\\rightarrow', '\\leftrightarrow'}:
                self.FB()

    def C(self):
        token = self.atual()
        if eh_constante(token):
            self.match(token)

    def P(self):
        token = self.atual()
        if eh_proposicao(token):
            self.match(self.atual())

    def FU(self):
        self.match('(')
        self.match('\\neg')
        self.F()
        self.match(')')

    def FB(self):
        self.match('(')
        op = self.atual()
        if op in {'\\wedge', '\\vee', '\\rightarrow', '\\leftrightarrow'}:
            self.match(op)
        self.F()
        self.F()
        self.match(')')

In [140]:
# Lê o arquivo testes.txt
caminho_arquivo = "/content/testes.txt"

with open(caminho_arquivo, "r", encoding="utf-8") as f:
    linhas = f.readlines()

qtd = int(linhas[0].strip())
expressoes = [linha.strip() for linha in linhas[1:]]

# Valida e imprime o resultado de cada expressão
for expr in expressoes:
    tokens = expr.replace("(", " ( ").replace(")", " ) ").split()

    fsm = StateMachine()

    result = fsm.process(tokens)
    if  result == False:
        print("invalida")
    else:
      parser = LL1_Parser(result)
      if parser.parse():
          print("valida")
      else:
          print("invalida")


valida
valida
valida
invalida
valida
valida
valida
valida
valida
valida
valida
valida
